In [1]:
import os

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/agbirhade1/MLOps-Project7.mlflow"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "bb8e44ff7f0afbc4148df5114797562149c14a25" #in dags hub this is data/dvc access key
os.environ["MLFLOW_TRACKING_USERNAME"] = "agbirhade1"

In [2]:
import os
%pwd



'c:\\Users\\theVa\\Documents\\GitHub\\mlops-course\\7-End2End\\research'

In [3]:

os.chdir('../')

In [4]:
from dataclasses import dataclass

@dataclass
class ModelEvaluationConfig:
    root_dir: str
    model_path: str
    test_data_path: str
    metric_file_name: str
    all_params: dict
    target_column: str
    mlflow_uri: str

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json

from src.datascience.entity.config_entity import ModelTrainerConfig,DataIngestionConfig,DataValidationConfig,DataTransformationConfig

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        create_directories([config.root_dir])
        model_val_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_path = config.model_path,
            test_data_path=config.test_data_path,
            metric_file_name=config.metric_file_name,
            all_params=params,
            target_column=self.schema.TARGET_COLUMN.name,
            mlflow_uri=os.environ["MLFLOW_TRACKING_URI"]
        )
        return model_val_config


In [6]:
import os
import pandas as pd
from src.datascience.utils.common import save_json
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        X_test = test_data.drop(columns=[self.config.target_column], axis=1)
        Y_test = test_data[self.config.target_column]
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_uri_store_type = urlparse(mlflow.get_tracking_uri()).scheme
        
        Y_pred = model.predict(X_test)
        rmse, mae, r2 = self.eval_metrics(Y_test, Y_pred)
        scores = {'rmse': rmse, 'mae': mae, 'r2': r2}
        save_json(path = Path(self.config.metric_file_name), data = scores)
        mlflow.log_params(self.config.all_params)

        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)

        if tracking_uri_store_type != "file":
            mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
        else:
            mlflow.sklearn.log_model(model, "model")

In [7]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    model_eval = ModelEvaluation(config=model_eval_config)  
    model_eval.log_into_mlflow()
except Exception as e:
    raise e

[2026-01-04 10:53:58,341: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2026-01-04 10:53:58,342: INFO: common: yaml file: params.yaml loaded successfully.]
[2026-01-04 10:53:58,345: INFO: common: yaml file: schema.yaml loaded successfully.]
[2026-01-04 10:53:58,345: INFO: common: Created directory at artifacts]
[2026-01-04 10:53:58,345: INFO: common: Created directory at artifacts/model_evaluation]
[2026-01-04 10:53:58,401: INFO: common: Saved json at artifacts\model_evaluation\metric.json]


c:\Users\theVa\Documents\GitHub\mlops-course\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'ElasticNetModel'.
2026/01/04 10:54:10 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.
